In [4]:
'''
- ลองใช้วิธีเช็คจาก timestamp จาก database 
- ยังไม่ได้พิจารณาการเก็บค่า เมื่อ ไฟดับ
- 
'''

                                                                                                                                                                      
import pandas as pd
import MySQLdb
import time
import datetime
import sys

from sympy import *
from sympy.solvers import solve
from sympy.parsing.sympy_parser import parse_expr

time_a = pd.date_range("00:00:00", "23:45:00", freq="15min")
cTime_a =  time_a.strftime("Timestamp('%Y-%m-%d %H:%M:%S')")
Ta = cTime_a.astype(str)

time_b = pd.date_range("00:14:59", "23:59:59", freq="15min")
cTime_b =  time_b.strftime("Timestamp('%Y-%m-%d %H:%M:%S')")
Tb = cTime_b.astype(str)

conn= MySQLdb.connect("localhost","conn","hems","power")
c = conn.cursor()

all_p = 0
allpower = []


# To check starting time j 
Hr = time.strftime('%H')
Mn = int(time.strftime('%M'))
Y = time.strftime('%Y')
if 0 < Mn <= 15:
    k = 0
elif 15 < Mn <= 30:
    k = 1
elif 30 < Mn <= 45:
    k = 2
else:
    k = 3
j = int(Hr)*4 + k


#To check the no of day in month 
dict = {'January': 31, 'February': 28, 'March': 31 ,
        'April': 30 ,'May': 31 ,'June': 30,
        'July': 31,'August': 31 ,'September': 30 ,
        'October': 31 ,'November': 30 , 'December': 31}   

turn = (dict[time.strftime('%B')]-int(time.strftime('%d')))+1

## Parametr of biil function
# x = raw_input('What about Ft?')
# Ft = float(x)
Ft = -0.048
t1 = 3.2484
t2 = 4.2218
t3 = 4.4217
ser = 38.22
u = symbols('u')

def bill(allpow):    
    ## Fundamental parameter
    if allpow == 0:
        bath = 0
    elif 1 <= allpow <= 150:
        bath = allpow*t1
    elif  151 <= allpow <= 400:
        power = allpow-150
        bath_1 = 150*t1
        bath_2 = power*t2
        bath = bath_1+bath_2
    else:
        power = allpow-150
        bath_1 = 150*t1
        power = power-250
        bath_2 = 250*t2
        bath_3 = power*t3
        bath = bath_1+bath_2+bath_3   
    service_price = 38.22  #permonth (fixed)
   
    ## FT -33.2 satang/unit
    FT_price = allpow*Ft

    ## VAT 7%
    VAT_price = ((service_price+bath)+FT_price)*0.07

    bill = bath+service_price+FT_price+VAT_price

    return round(bill,2)

    

while(True):
    clock = time.strftime(u"Timestamp('%Y-%m-%d %H:%M:%S')")
#     print clock
    min_p = 0  
    
    if (j == 96): 
        allpower.append(all_p)
        time_a = pd.date_range("00:00:00", "23:45:00", freq="15min")
        Ta = time_a.strftime("Timestamp('%Y-%m-%d %H:%M:%S')").astype(str)
        Tb = pd.date_range("00:14:59", "23:59:59", freq="15min").strftime("Timestamp('%Y-%m-%d %H:%M:%S')").astype(str)
        j = 0

        if len(allpower) == turn:
            sum_allpower = sum(int(i) for i in allpower)
            bill(sum_allpower)
            this_month = time.strftime('%B')
            print "the electrical bill of",this_month,Y,"is ::",bill(all_p/1000),"bath ::"  
            allpower = []  
            time.sleep(1)
            turn = dict[time.strftime('%B')]
            print "next bill in",turn,"day\n\n\n"    
            all_p = 0           
            

    if (clock == Tb [j]):
        print "\n\n\n",clock ,"....... process ....... \n"
        query = "SELECT Timestamp,s11 FROM power.enegate_bkk WHERE Timestamp >= '%s'   AND TimeStamp <= '%s'; "% (time_a[j],time_b[j])
        df = pd.read_sql(query, conn)
        df = df[df.s11 != "----"]
        df = df.reset_index(drop=True)
        df['s11'] = df['s11'].astype(float)
#         print "..///////////////////////////////////////////////////////////////////////.."
        if len(df) == 0 :
            min_p = 0
        else :
            for i in range(len(df)):
                min_p += df.s11[i]        
            min_p = min_p/len(df)/4  
#         all_p += min_p
        print  "You use electrical energy in 15 minute left -->",min_p/1000,"KW/15min that equal to",bill(min_p/1000),"bath"
        all_p += 1000
        print  "the electrical bill of",time_a[0],"to NOW. equal to",bill(all_p/1000),"bath\n"  
        j += 1
            
    time.sleep(1)
    print("."),
    
        


 . . . . . . . . 


Timestamp('2016-11-30 23:44:59') process ....... 

You use electrical energy in 15 minute left --> 0 KW/15min that equal to 40.9 bath
the electrical bill of 2016-11-30 00:00:00 to NOW. equal to 44.32 bath

. . . . . . . . . . . . . . . . . . . 


Timestamp('2016-11-30 23:59:59') process ....... 

You use electrical energy in 15 minute left --> 0 KW/15min that equal to 40.9 bath
the electrical bill of 2016-11-30 00:00:00 to NOW. equal to 47.74 bath

. the electrical bill of December 2016 is :: 47.74 bath ::



next bill in 31 day
. . . . . . . . . . . . . . . . . . . . . . . 


Timestamp('2016-12-01 00:14:59') process ....... 

You use electrical energy in 15 minute left --> 0 KW/15min that equal to 40.9 bath
the electrical bill of 2016-12-01 00:00:00 to NOW. equal to 44.32 bath

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 


Timestamp('2016-12-01 00:29:59') process ....... 

You use electrical energy in 15 minute left --> 0 KW/15min that equal to 40.

KeyboardInterrupt: 

In [ ]:

Tb = pd.date_range("00:14:59", "23:59:59", freq="15min").strftime("Timestamp('%Y-%m-%d %H:%M:%S')").astype(str)
print Tb

